### Analytical solution

The code below is to check our work on what the analytical solution of the 1D Stefan problem is.

In [ ]:
import sympy
from sympy import exp, simplify, integrate

Make some symbols representing the thickness, vertical coordinate, etc.
Our proposed solution for the energy function involves some multiple integrals, so we have to introduce symbols not just for the vertical coordinate $\zeta$, but also some auxiliary integration variables $\zeta_1$, $\zeta_2$.

In [ ]:
h, κ, ζ, ζ_1, ζ_2, ζ_m, E_m, E_s = sympy.symbols(
    "h κ ζ ζ_1 ζ_2 ζ_m E_m E_s", real=True, positive=True
)

We separately introduce a symbol for the vertical velocity $\omega$.
All of the previous symbols are positive; here we only assume that the vertical velocity is non-zero.

In [ ]:
ω = sympy.symbols("ω", real=True, nonzero=True)

Next, we introduce the heating term $Q$ as a symbolic uninterpreted function of $\zeta$.

In [ ]:
Q = sympy.Function("Q", real=True, positive=True)(ζ)

Next we introduce the integrating factor $\phi$ and an auxiliary quantity $W$ which will make the later definition of $E$ much shorter.

In [ ]:
ϕ = h**2 * ω / κ * (ζ - ζ_m)

In [ ]:
W = h * integrate(Q.subs({ζ: ζ_1}), (ζ_1, ζ, 1)) - h * ω * E_s
W

Our proposed expression for the energy function, obtained from the method of integrating factors.

In [ ]:
E = simplify(
    exp(ϕ) * E_m + (h / κ) * exp(ϕ) * integrate((exp(-ϕ) * W).subs({ζ: ζ_2}), (ζ_2, ζ_m, ζ))
)
E

Let's check if this function actually is a solution of the boundary-value problem.
First, we'll evaluate the differential operator on it.
If we did this correctly, we should just get $h\cdot Q$ as the result.

In [ ]:
simplify(h * ω * E - κ / h * E.diff(ζ)).diff(ζ)

Now let's see if the surface boundary condition is correct.
If we did this right, we should just get back $f_s$.

In [ ]:
simplify((h * ω * E - κ / h * E.diff(ζ)).subs({ζ: 1}))

Now let's see what the conductive heat flux is at the transition depth $\zeta_m$.
The right value of $\zeta_m$ makes this quantity equal to zero.

In [ ]:
residual = simplify((κ / h * E.diff(ζ)).subs({ζ: ζ_m}))
residual

If the volumetric heating is simple enough, then we might be able to solve explicitly for the transition depth.
For example, for a power-law fluid in simple $xz$-shear, the strain heating rate has the form
$$Q = Q_0(1 - \zeta)^{n + 1}$$
where $n = 1$ for a Newtonian fluid.
Then the equation to be solved is
$$\begin{align}
0 & = \omega\Delta E + \int_{\zeta_m}^1Q(\zeta)d\zeta \\
& = \omega\Delta E - Q_0\frac{(1 - \zeta)^{n + 2}}{n + 2}\Big|_{\zeta_m}^1 \\
& = \omega\Delta E + \frac{Q_0}{n + 2}(1 - \zeta_m)^{n + 2}
\end{align}$$
and so, remembering again that $\omega$ is negative,
$$
\zeta_m = 1 - \left(-(n + 2)\frac{\omega\Delta E}{Q_0}\right)^{1/(n + 2)}.
$$
Moreover, we can see immediately that $Q_0$ has to exceed $(n + 2)\omega\Delta E$ in order to have a cold-temperate transition within the fluid column.

### Numerical solution

Now we'll try to solve the boundary value problem numerically and check that it matches the exact solution.
The example in the Greve and Blatter book uses the following physical parameters.

| name | symbol | value
| ---- | ---- | ----
| thickness | $h$ | 200 m
| density | $\rho$ | 917 kg m${}^{-3}$
| heat capacity | $c_p$ | 2.09 kJ kg${}^{-1}$ ${}^\circ$C${}^{-1}$
| diffusivity | $\kappa$ | 32.16 m${}^2$ yr${}^{-1}$
| vertical velocity | $u_z$ | 0.2 m yr${}^{-1}$
| surface temperature | $T_s$ | -3 ${}^\circ$C
| surface slope | $\gamma$ | 4${}^\circ$
| ice fluidity | $A$ | 8.02 $\times$ 10${}^{-8}$ yr${}^{-1}$ kPa ${}^{-3}$

In [ ]:
import numpy as np
from numpy import pi as π
import matplotlib.pyplot as plt
import tqdm
import firedrake
from firedrake import max_value, min_value, Constant, conditional, inner, grad, dx, ds
import irksome
from irksome import Dt

In [ ]:
year = 365.25 * 24 * 60 * 60
n = 3
ρ = Constant(917.0)                # kg / m^3
c_p = Constant(2.09)               # kJ / kg / C
κ = Constant(32.16)                # m^2 / yr
g = Constant(9.81)                 # m / s^2
h = Constant(200.0)                # m
Q = 139                            # kJ / mol
R = 8.314e-3                       # kJ / mol K
T = 273.15 - 3                     # K
A_0 = 1.916e3 * year * 1e9         # 1 / yr / kPa^3
A = Constant(A_0 * np.exp(-Q / (R * T)))
θ = 4
γ = Constant(np.sin(2 * π * θ / 360))
u_z = Constant(0.2)                # m / yr

In [ ]:
τ = (ρ * g * h / 1e3) * γ
print(f"Driving stress: {float(τ):.2f} kPa")

In [ ]:
Q_0 = 2 * A * τ**(n + 1)
print(f"Max strain heating: {float(Q_0):.2f} kJ / m^3 / yr")

In [ ]:
T_m = 273.15
ΔE = ρ * c_p * (T_m - T)
ω = u_z / h
print(f"Advection / production ratio: {float((n + 2) * ΔE * ω / Q_0):.2f}")

In [ ]:
ζ_m = float(1 - ((n + 2) * ΔE * ω / Q_0) ** (1 / (n + 2)))
print(f"Height of CTS: {ζ_m:.2f}")

In [ ]:
def energy_form(**kwargs):
    field_names = (
        "energy",
        "test_function",
        "thickness",
        "velocity",
        "sources",
        "energy_inflow",
        "diffusivity",
    )
    E, ϕ, h, u, Q, E_in, κ = map(kwargs.get, field_names)

    F = h * E * u - κ / h * grad(E)
    G_cells = (h * Dt(E) * ϕ - inner(F, grad(ϕ)) - h * Q * ϕ) * dx
    n = firedrake.FacetNormal(mesh)
    G_outflow = h * E * max_value(0, inner(u, n)) * ϕ * ds
    G_inflow = h * E_in * min_value(0, inner(u, n)) * ϕ * ds
    return G_cells + G_outflow + G_inflow

In [ ]:
u = firedrake.as_vector((-u_z,)) / h  # 1 / yr

In [ ]:
nx = 128
mesh = firedrake.UnitIntervalMesh(nx)
degree = 1
element = firedrake.FiniteElement("CG", "interval", degree)
temperature_space = firedrake.FunctionSpace(mesh, element)

In [ ]:
ζ = firedrake.SpatialCoordinate(mesh)[0]
E_s = ρ * c_p * (T - T_m)
E = firedrake.Function(temperature_space)
ϕ = firedrake.TestFunction(temperature_space)
E.interpolate(ζ * E_s)
fields = {
    "energy": E,
    "test_function": ϕ,
    "thickness": h,
    "velocity": u,
    "sources": Q_0 * (1 - ζ) ** (n + 1),
    "energy_inflow": E_s,
    "diffusivity": firedrake.conditional(E <= 0.0, κ, 0),
}
F = energy_form(**fields)

In [ ]:
t = Constant(0.0)
dt = Constant(0.25)
final_time = 1000.0
num_steps = int(final_time / float(dt))
Es = [E.copy(deepcopy=True)]
method = irksome.BackwardEuler()
params = {
    "solver_parameters": {
        "snes_max_it": 5,
        "snes_convergence_test": "skip",
        "ksp_type": "gmres",
    },
}
solver = irksome.TimeStepper(F, method, t, dt, E, **params)
for step in tqdm.trange(num_steps):
    solver.advance()
    Es.append(E.copy(deepcopy=True))

In [ ]:
fig, ax = plt.subplots()
ax.set_ylim((-6e3, 1e3))
firedrake.plot(Es[10], axes=ax)
firedrake.plot(Es[200], axes=ax)
firedrake.plot(Es[2000], axes=ax)
firedrake.plot(Es[-1], axes=ax);

In [ ]:
L = Constant(334)
W = firedrake.Function(temperature_space).interpolate(firedrake.max_value(0, E) / (ρ * L))

In [ ]:
fig, ax = plt.subplots()
firedrake.plot(W, axes=ax);